In [1]:
from autotst.reaction import *
from autotst.molecule import *
from autotst.geometry import *

thermo.py:839 loadLibraries INFO Loading thermodynamics library from primaryThermoLibrary.py in /home/harms.n/Code/RMG-master/../RMG-database/input/thermo/libraries...
thermo.py:839 loadLibraries INFO Loading thermodynamics library from thermo_DFT_CCSDTF12_BAC.py in /home/harms.n/Code/RMG-master/../RMG-database/input/thermo/libraries...
thermo.py:839 loadLibraries INFO Loading thermodynamics library from CBS_QB3_1dHR.py in /home/harms.n/Code/RMG-master/../RMG-database/input/thermo/libraries...
thermo.py:856 loadGroups INFO Loading thermodynamics group database from /home/harms.n/Code/RMG-master/../RMG-database/input/thermo/groups...
transport.py:294 loadGroups INFO Loading transport group database from /home/harms.n/Code/RMG-master/../RMG-database/input/transport/groups...
database.py:167 loadFamilies INFO Loading the user-specified kinetics families from /home/harms.n/Code/RMG-master/../RMG-database/input/kinetics/families
statmech.py:526 loadGroups INFO Loading frequencies group data

In [2]:
reactants = [Molecule(SMILES="CCCC"), Molecule(SMILES="[O]O")]
products = [Molecule(SMILES="[CH2]CCC"), Molecule(SMILES="OO")]
rmg_reaction = Reaction(reactants=reactants, products=products)
rmg_reaction

In [3]:
test_reaction = AutoTST_Reaction(reaction_family="H_Abstraction", rmg_reaction=rmg_reaction)
test_reaction

reaction.py:233 get_rmg_reactions INFO The distance data is as follows: 
DistanceData(distances={'d12': 1.380401,'d13': 2.530343,'d23': 1.157074,}, uncertainties={'d12': 0.385038,'d13': 0.208449,'d23': 0.303191,}, comment=u'Matched node C/H3/Cs ([<Entry index=118 label="C/H3/Cs">, <Entry index=262 label="Cdj_CdsCs">])\nMatched node OjO ([<Entry index=126 label="C/H2/Cd/Cd">, <Entry index=32 label="OjO">])\n')
reaction.py:351 get_labels INFO The labled atoms are [2, 7, 0].
reaction.py:296 create_rdkit_ts_geometry INFO Initially embedded molecule
reaction.py:299 create_rdkit_ts_geometry INFO Editing bounds matrix
reaction.py:366 set_limits INFO For atoms 2 and 7 we have a distance of: 	 1.380401
reaction.py:366 set_limits INFO For atoms 7 and 0 we have a distance of: 	 1.157074
reaction.py:366 set_limits INFO For atoms 2 and 0 we have a distance of: 	 2.530343
reaction.py:421 bm_pre_edit INFO Changing lower limit 2.520343 to 2.457475
reaction.py:421 bm_pre_edit INFO Changing lower limit 

<autotst.reaction.AutoTST_Reaction instance at 0x2ba50c22e7a0>

In [4]:
from ase.calculators.gaussian import *

In [5]:
def rxn_shell_calc(autotst_rxn, scratch):
    
    indicies = []
    for i, atom in enumerate(autotst_rxn.ts.rmg_ts.atoms):
        if not (atom.label == ""):
            indicies.append(i)
    
    combos = []
    for combo in list(itertools.combinations(indicies, 2)):
        a,b = combo
        combos.append("{0} {1} F".format(a,b))
    

    autotst_rxn.ts.rmg_ts.updateMultiplicity()
    
    calc = Gaussian(mem="5GB", 
                    nprocshared="20", 
                    label= autotst_rxn.label + "_shell", 
                    scratch=scratch, 
                    method="m062x", 
                    basis="6-311+g(2df,2p)",
                    extra="opt=(ts,calcfc,noeigentest) freq",
                    multiplicity = test_reaction.ts.rmg_ts.multiplicity,
                    addsec = combos)
    
    del calc.parameters['force']
    
    autotst_rxn.ts.ase_ts.set_calculator(calc)
    return autotst_rxn


def rxn_center_calc(autotst_rxn, scratch):
    
    indicies = []
    for i, atom in enumerate(autotst_rxn.ts.rmg_ts.atoms):
        if (atom.label == ""):
            indicies.append(i)
            
    combos = []
    for combo in list(itertools.combinations(indicies, 2)):
        a,b = combo
        combos.append("{0} {1} F".format(a,b))
    
    autotst_rxn.ts.rmg_ts.updateMultiplicity()
    
    calc = Gaussian(mem="5GB", 
                    nprocshared="20", 
                    label=autotst_rxn.label + "_center", 
                    scratch=scratch, 
                    method="m062x", 
                    basis="6-311+g(2df,2p)",
                    extra="opt=(ts,calcfc,noeigentest)",
                    multiplicity = test_reaction.ts.rmg_ts.multiplicity,
                    addsec = combos)
    del calc.parameters['force']
    
    autotst_rxn.ts.ase_ts.set_calculator(calc)
    
    return autotst_rxn



def irc_calc(autotst_rxn, scratch):
    
    autotst_rxn.ts.rmg_ts.updateMultiplicity()
    
    calc = Gaussian(mem="5GB", 
                    nprocshared="20", 
                    label= autotst_rxn.label + "_irc", 
                    scratch=scratch, 
                    method="m062x", 
                    basis="6-311+g(2df,2p)",
                    extra="irc=(calcall)",
                    multiplicity = test_reaction.ts.rmg_ts.multiplicity)
    del calc.parameters['force']
    
    autotst_rxn.ts.ase_ts.set_calculator(calc)
    
    return autotst_rxn
    


In [6]:
rxn = rxn_shell_calc(test_reaction, ".")
calc1 = rxn.ts.ase_ts.get_calculator()
print calc1.label

CCCC+[O]O_[CH2]CCC+OO_shell


In [7]:
rxn = rxn_center_calc(test_reaction, ".")
calc2 = rxn.ts.ase_ts.get_calculator()
print calc2.label

CCCC+[O]O_[CH2]CCC+OO_center


In [8]:
rxn = irc_calc(test_reaction, ".")
calc3 = rxn.ts.ase_ts.get_calculator()
print calc3.label

CCCC+[O]O_[CH2]CCC+OO_irc


In [9]:
calc1.calculate(test_reaction.ts.ase_ts)

RuntimeError: Gaussian in . returned an error: 127